In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import time
import pymongo

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://ark.intel.com/content/www/us/en/ark.html#@Processors'
baseurl = 'https://ark.intel.com/'

browser.visit(url)

rootHTML = bs(browser.html, 'html.parser')

In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [5]:
db = client.IntelScrape
collection = db.items

In [6]:
for div in rootHTML.find_all('div', 'Processors'):
    for link in rootHTML.\
        find_all('div', {'data-parent-panel-key':div['data-panel-key']})[0].\
        find_all('a'):
        browser.visit(baseurl+link['href'])
        time.sleep(1)
        for td in bs(browser.html, 'html.parser').find_all('td', class_='ark-product-name'):
            browser.visit(baseurl+td.a['href'])
            time.sleep(1)
            data = {}
            for c in bs(browser.html, 'html.parser').\
                find_all('div', class_='arkProductSpecifications')[0].\
                find_all('span', class_=['label','value']):
                if c['class'][0] == 'label':
                    temp = c.text.strip().replace('.','_')
                else:
                    data[temp] = c.text.strip()
            collection.insert_one(data)
            print(td.a.text)

Intel® Core™ i9-10980XE Extreme Edition Processor
Intel® Core™ i9-10940X X-series Processor
Intel® Core™ i9-10920X X-series Processor
Intel® Core™ i9-10900X X-series Processor
Intel® Core™ i9-9980XE Extreme Edition Processor
Intel® Core™ i9-9960X X-series Processor
Intel® Core™ i9-9940X X-series Processor
Intel® Core™ i9-9920X X-series Processor
Intel® Core™ i9-9900X X-series Processor
Intel® Core™ i9-9820X X-series Processor
Intel® Core™ i7-9800X X-series Processor
Intel® Core™ i9-7980XE Extreme Edition Processor
Intel® Core™ i9-7960X X-series Processor
Intel® Core™ i9-7940X X-series Processor
Intel® Core™ i9-7920X X-series Processor
Intel® Core™ i9-7900X X-series Processor
Intel® Core™ i7-7820X X-series Processor
Intel® Core™ i7-7800X X-series Processor
Intel® Core™ i7-7740X X-series Processor
Intel® Core™ i5-7640X X-series Processor
Intel® Core™ i7-6950X Processor Extreme Edition
Intel® Core™ i7-6900K Processor
Intel® Core™ i7-6850K Processor
Intel® Core™ i7-6800K Processor
Intel® C

Intel® Core™ i5-6600 Processor
Intel® Core™ i5-6685R Processor
Intel® Core™ i5-6600K Processor
Intel® Core™ i5-6600T Processor
Intel® Core™ i5-6585R Processor
Intel® Core™ i5-6500 Processor
Intel® Core™ i5-6500T Processor
Intel® Core™ i5-6500TE Processor
Intel® Core™ i5-6402P Processor
Intel® Core™ i5-6400 Processor
Intel® Core™ i5-6400T Processor
Intel® Core™ i5-6440EQ Processor
Intel® Core™ i5-6440HQ Processor
Intel® Core™ i5-6442EQ Processor
Intel® Core™ i5-6360U Processor
Intel® Core™ i5-6350HQ Processor
Intel® Core™ i5-6300HQ Processor
Intel® Core™ i5-6300U Processor
Intel® Core™ i5-6200U Processor
Intel® Core™ i5-6260U Processor
Intel® Core™ i5-6267U Processor
Intel® Core™ i5-6287U Processor
Intel® Core™ i3-6300 Processor
Intel® Core™ i3-6300T Processor
Intel® Core™ i3-6320 Processor
Intel® Core™ i3-6100E Processor
Intel® Core™ i3-6100H Processor
Intel® Core™ i3-6100U Processor
Intel® Core™ i3-6102E Processor
Intel® Core™ i3-6157U Processor
Intel® Core™ i3-6167U Processor
Intel® 

Intel® Core™ i7-3630QM Processor
Intel® Core™ i7-3615QM Processor
Intel® Core™ i7-3615QE Processor
Intel® Core™ i7-3612QM Processor
Intel® Core™ i7-3612QM Processor
Intel® Core™ i7-3612QE Processor
Intel® Core™ i7-3610QM Processor
Intel® Core™ i7-3610QE Processor
Intel® Core™ i7-3555LE Processor
Intel® Core™ i7-3540M Processor
Intel® Core™ i7-3537U Processor
Intel® Core™ i7-3520M Processor
Intel® Core™ i7-3517UE Processor
Intel® Core™ i7-3517U Processor
Intel® Core™ i5-3610ME Processor
Intel® Core™ i5-3570S Processor
Intel® Core™ i5-3570T Processor
Intel® Core™ i5-3570K Processor
Intel® Core™ i5-3570 Processor
Intel® Core™ i5-3550S Processor
Intel® Core™ i5-3550 Processor
Intel® Core™ i5-3475S Processor
Intel® Core™ i5-3470T Processor
Intel® Core™ i5-3470S Processor
Intel® Core™ i5-3470 Processor
Intel® Core™ i5-3450S Processor
Intel® Core™ i5-3450 Processor
Intel® Core™ i5-3439Y Processor
Intel® Core™ i5-3437U Processor
Intel® Core™ i5-3427U Processor
Intel® Core™ i5-3380M Processor
I

Intel® Core™2 Duo Processor P9700
Intel® Core™2 Duo Processor P9600
Intel® Core™2 Duo Processor P9500
Intel® Core™2 Duo Processor P8800
Intel® Core™2 Duo Processor P8700
Intel® Core™2 Duo Processor P8600
Intel® Core™2 Duo Processor P8400
Intel® Core™2 Duo Processor P7570
Intel® Core™2 Duo Processor P7550
Intel® Core™2 Duo Processor P7450
Intel® Core™2 Duo Processor P7370
Intel® Core™2 Duo Processor P7350
Intel® Core™2 Duo Processor SL9300
Intel® Core™2 Duo Processor SL9600
Intel® Core™2 Duo Processor SL9400
Intel® Core™2 Duo Processor SL9380
Intel® Core™2 Duo Processor SP9600
Intel® Core™2 Duo Processor SP9400
Intel® Core™2 Duo Processor SP9300
Intel® Core™2 Duo Processor SU9600
Intel® Core™2 Duo Processor SU9400
Intel® Core™2 Duo Processor SU9300
Intel® Core™2 Duo Processor T9900
Intel® Core™2 Duo Processor T9800
Intel® Core™2 Duo Processor T9600
Intel® Core™2 Duo Processor T9550
Intel® Core™2 Duo Processor T9500
Intel® Core™2 Duo Processor T9400
Intel® Core™2 Duo Processor T9300
Inte

Intel® Pentium® 4 Processor 662 supporting HT Technology
Intel® Pentium® 4 Processor 661 supporting HT Technology
Intel® Pentium® 4 Processor 660 supporting HT Technology
Intel® Pentium® 4 Processor 651 supporting HT Technology
Intel® Pentium® 4 Processor 650 supporting HT Technology
Intel® Pentium® 4 Processor 641 supporting HT Technology
Intel® Pentium® 4 Processor 640 supporting HT Technology
Intel® Pentium® 4 Processor 631 supporting HT Technology
Intel® Pentium® 4 Processor 630 supporting HT Technology
Intel® Pentium® 4 Processor 620 supporting HT Technology
Intel® Pentium® 4 Processor 571 supporting HT Technology
Intel® Pentium® 4 Processor 570J supporting HT Technology
Intel® Pentium® 4 Processor 561 supporting HT Technology
Intel® Pentium® 4 Processor 560J supporting HT Technology
Intel® Pentium® 4 Processor 560/560J supporting HT Technology
Intel® Pentium® 4 Processor 551 supporting HT Technology
Intel® Pentium® 4 Processor 550J supporting HT Technology
Intel® Pentium® 4 Proce

...

Mobile Intel® Celeron® Processor 650 MHz, 128K Cache, 100 MHz FSB
Mobile Intel® Celeron® Processor 600 MHz, 128K Cache, 100 MHz FSB
Mobile Intel® Celeron® Processor 550 MHz, 128K Cache, 100 MHz FSB
Mobile Intel® Celeron® Processor 500 MHz, 128K Cache, 100 MHz FSB
Mobile Intel® Celeron® Processor 466 MHz, 128K Cache, 66 MHz FSB
Mobile Intel® Celeron® Processor 450 MHz, 128K Cache, 100 MHz FSB
Mobile Intel® Celeron® Processor 433 MHz, 128K Cache, 66 MHz FSB
Mobile Intel® Celeron® Processor LV 400 MHz, 128K Cache, 100 MHz FSB
Mobile Intel® Celeron® Processor 400 MHz, 128K Cache, 66 MHz FSB
Mobile Intel® Celeron® Processor 366 MHz, 128K Cache, 66 MHz FSB
Mobile Intel® Celeron® Processor 300 MHz, 128K Cache, 66 MHz FSB
Mobile Intel® Celeron® Processor 266 MHz, 128K Cache, 66 MHz FSB
Intel® Celeron® Processor 2.80 GHz, 128K Cache, 400 MHz FSB
Intel® Celeron® Processor 2.70 GHz, 128K Cache, 400 MHz FSB
Intel® Celeron® Processor 2.60 GHz, 128K Cache, 400 MHz FSB
Intel® Celeron® Processor 2.50 

Intel® Xeon® Processor E7-8891 v2
Intel® Xeon® Processor E7-8893 v2
Intel® Xeon® Processor E7-4890 v2
Intel® Xeon® Processor E7-4880 v2
Intel® Xeon® Processor E7-4870 v2
Intel® Xeon® Processor E7-4860 v2
Intel® Xeon® Processor E7-4850 v2
Intel® Xeon® Processor E7-4830 v2
Intel® Xeon® Processor E7-4820 v2
Intel® Xeon® Processor E7-4809 v2
Intel® Xeon® Processor E7-2890 v2
Intel® Xeon® Processor E7-2880 v2
Intel® Xeon® Processor E7-2870 v2
Intel® Xeon® Processor E7-2850 v2
Intel® Xeon® Processor E7-8870
Intel® Xeon® Processor E7-8867L
Intel® Xeon® Processor E7-8860
Intel® Xeon® Processor E7-8850
Intel® Xeon® Processor E7-8837
Intel® Xeon® Processor E7-8830
Intel® Xeon® Processor E7-4870
Intel® Xeon® Processor E7-4860
Intel® Xeon® Processor E7-4850
Intel® Xeon® Processor E7-4830
Intel® Xeon® Processor E7-4820
Intel® Xeon® Processor E7-4807
Intel® Xeon® Processor E7-2870
Intel® Xeon® Processor E7-2860
Intel® Xeon® Processor E7-2850
Intel® Xeon® Processor E7-2830
Intel® Xeon® Processor E7-2

Intel® Xeon® Processor E3-1240L v5
Intel® Xeon® Processor E3-1240 v5
Intel® Xeon® Processor E3-1235L v5
Intel® Xeon® Processor E3-1230 v5
Intel® Xeon® Processor E3-1225 v5
Intel® Xeon® Processor E3-1220 v5
Intel® Xeon® Processor E3-1285L v4
Intel® Xeon® Processor E3-1285 v4
Intel® Xeon® Processor E3-1278L v4
Intel® Xeon® Processor E3-1265L v4
Intel® Xeon® Processor E3-1258L v4
Intel® Xeon® Processor E3-1286L v3
Intel® Xeon® Processor E3-1286 v3
Intel® Xeon® Processor E3-1285L v3
Intel® Xeon® Processor E3-1285 v3
Intel® Xeon® Processor E3-1281 v3
Intel® Xeon® Processor E3-1280 v3
Intel® Xeon® Processor E3-1276 v3
Intel® Xeon® Processor E3-1275L v3
Intel® Xeon® Processor E3-1270 v3
Intel® Xeon® Processor E3-1268L v3
Intel® Xeon® Processor E3-1271 v3
Intel® Xeon® Processor E3-1275 v3
Intel® Xeon® Processor E3-1265L v3
Intel® Xeon® Processor E3-1246 v3
Intel® Xeon® Processor E3-1245 v3
Intel® Xeon® Processor E3-1241 v3
Intel® Xeon® Processor E3-1240L v3
Intel® Xeon® Processor E3-1240 v3
In

Intel® Xeon® Processor L5408
Intel® Xeon® Processor E5405
Intel® Xeon® Processor X5365
Intel® Xeon® Processor X5355
Intel® Xeon® Processor E5345
Intel® Xeon® Processor L5335
Intel® Xeon® Processor E5335
Intel® Xeon® Processor L5320
Intel® Xeon® Processor E5320
Intel® Xeon® Processor L5318
Intel® Xeon® Processor L5310
Intel® Xeon® Processor E5310
Intel® Xeon® Processor X5272
Intel® Xeon® Processor X5270
Intel® Xeon® Processor X5260
Intel® Xeon® Processor L5240
Intel® Xeon® Processor E5240
Intel® Xeon® Processor L5238
Intel® Xeon® Processor E5220
Intel® Xeon® Processor L5215
Intel® Xeon® Processor E5205
Intel® Xeon® Processor 5160
Intel® Xeon® Processor 5150
Intel® Xeon® Processor LV 5148
Intel® Xeon® Processor 5140
Intel® Xeon® Processor LV 5138
Intel® Xeon® Processor LV 5133
Intel® Xeon® Processor 5130
Intel® Xeon® Processor LV 5128
Intel® Xeon® Processor 5120
Intel® Xeon® Processor LV 5113
Intel® Xeon® Processor 5110
Intel® Xeon® Processor 5080
Intel® Xeon® Processor 5070
Intel® Xeon®

Intel® Xeon Phi™ Processor 7210F
Intel® Xeon Phi™ Processor 7210
Intel® Xeon Phi™ Processor 7295
Intel® Xeon Phi™ Processor 7285
Intel® Xeon Phi™ Processor 7235
Intel® Itanium® Processor 9720
Intel® Itanium® Processor 9740
Intel® Itanium® Processor 9750
Intel® Itanium® Processor 9760
Intel® Itanium® Processor 9560
Intel® Itanium® Processor 9550
Intel® Itanium® Processor 9540
Intel® Itanium® Processor 9520
Intel® Itanium® Processor 9350
Intel® Itanium® Processor 9340
Intel® Itanium® Processor 9330
Intel® Itanium® Processor 9320
Intel® Itanium® Processor 9310
Intel® Itanium® Processor 9152M
Intel® Itanium® Processor 9150N
Intel® Itanium® Processor 9150M
Intel® Itanium® Processor 9140N
Intel® Itanium® Processor 9140M
Intel® Itanium® Processor 9130M
Intel® Itanium® Processor 9120N
Intel® Itanium® Processor 9110N
Intel® Itanium® Processor 9050
Intel® Itanium® Processor 9040
Intel® Itanium® Processor 9030
Intel® Itanium® Processor 9020
Intel® Itanium® Processor 9015
Intel® Itanium® Processor

In [7]:
browser.quit()

In [8]:
import csv
with open('IntelArk.csv','w',newline='') as csvfile:
    cols = []
    data = db.items.find()
    for i in data:
        cols.extend([x for x in i.keys() if x not in cols])
    cols.remove('_id')
    
    writer = csv.DictWriter(csvfile, cols)
    writer.writeheader()
    for i in data.rewind():
        i.pop('_id')
        writer.writerow(i)